In [62]:
import ee
import os
import geemap

S2_BANDS = ['B2', 'B3', 'B4', 'B8',  # 10m
            'B5', 'B8A', 'B11']  # 20m

PROJECT_NAME = 'treeai-470815'
BUCKET_NAME = 'treeai_data'

# Initialize the Earth Engine module and register this code in project 'TreeAI'
ee.Authenticate()
ee.Initialize(project=PROJECT_NAME)

In [64]:
test_img = ee.Image('projects/treeai-470815/assets/20220518_1049_12504_0_15_p_14848_2560')

In [128]:
def generate_monthly_SwissEO(target_geometry, start_date, end_date, s2_bands):
    col = (ee.ImageCollection("projects/satromo-prod/assets/col/S2_SR_HARMONIZED_SWISS")
           .filterBounds(target_geometry)
           .filterDate(start_date, end_date))

    images_10m = col.filter(ee.Filter.eq('pixel_size_meter', 10)).sort('system:time_start', True)
    images_20m = col.filter(ee.Filter.eq('pixel_size_meter', 20)).sort('system:time_start', True)

    n = images_10m.size()
    l10 = images_10m.toList(n)
    l20 = images_20m.toList(n)

    def combine_pair(i):
        i   = ee.Number(i)
        i10 = ee.Image(l10.get(i))
        i20 = ee.Image(l20.get(i))
        return (ee.Image.cat([
                    i10.select(['B2', 'B3', 'B4', 'B8', 'cloudAndCloudShadowMask']),
                    i20.select(['B5', 'B8A', 'B11'])
                ])
                .copyProperties(i10, ['system:time_start', 'system:index']))

    merged = ee.ImageCollection(ee.List.sequence(0, n.subtract(1)).map(combine_pair))

    def downsample_mask_band(image):
        other_bands = image.select(S2_BANDS)
        mask_band = image.select('cloudAndCloudShadowMask').resample('nearest').reproject({'crs': img.projection(), 'scale': 20})
        mask_band =

    return merged


In [130]:
start_date = ee.Date.fromYMD(2019, 5, 1)
end_date = start_date.advance(1, 'month')
swiss_img = generate_monthly_SwissEO(target_geometry=test_img.geometry(), start_date=start_date, end_date=end_date, s2_bands=S2_BANDS)
swiss_img.first()